In [22]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score as auc, accuracy_score
import numpy as np

In [23]:
n = 80
df = pd.read_csv('../prepare data/data/base_line.csv')
df.drop('match_id', axis=1, inplace=True)
df.head()

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
0,20,2023,West Ham United,Brighton & Hove Albion,London Stadium,2
1,20,2023,Liverpool,Newcastle United,Anfield,1
2,20,2023,Fulham,Arsenal,Craven Cottage,1
3,20,2023,Tottenham Hotspur,Bournemouth,Tottenham Hotspur Stadium,1
4,20,2023,Nottingham Forest,Manchester United,The City Ground,1


In [24]:
train = df.iloc[n:, :]
val = df.iloc[:n, :]

In [25]:
train

,gameweek_gameweek,gameweek_compSeason_label,teams_team_1_name,teams_team_2_name,ground_name,team_1_hue
80,12,2023,Brighton & Hove Albion,Sheffield United,American Express Stadium,2
81,12,2023,Liverpool,Brentford,Anfield,1
82,12,2023,West Ham United,Nottingham Forest,London Stadium,1
83,12,2023,Bournemouth,Newcastle United,Vitality Stadium,1
84,12,2023,Arsenal,Burnley,Emirates Stadium,1
...,...,...,...,...,...,...
1713,1,2019,Burnley,Southampton,Turf Moor,1
1714,1,2019,Crystal Palace,Everton,Selhurst Park,2
1715,1,2019,Watford,Brighton & Hove Albion,Vicarage Road,0
1716,1,2019,West Ham United,Manchester City,London Stadium,0


In [26]:
y_train = train['team_1_hue']
x_train = train.drop('team_1_hue', axis=1)

y_val = val['team_1_hue']
x_val = val.drop('team_1_hue', axis=1)

In [27]:
model_baseline = CatBoostClassifier()

In [28]:
model_baseline.fit(x_train, y_train, cat_features=list(x_train.columns), verbose=False)

In [29]:
pred = model_baseline.predict(x_val)
pred_proba = model_baseline.predict_proba(x_val)
y_val = y_val.to_numpy().reshape(-1, 1)

In [30]:
pred = np.concatenate((y_val, pred, pred_proba), axis=1)
df_pred = pd.DataFrame(pred, columns=['true', 'pred', '0_prob', '1_prob', '2_prob'])

In [31]:
df_pred['error'] = np.where(df_pred['true']==df_pred['pred'], 0, 1)
df_pred.head()


,true,pred,0_prob,1_prob,2_prob,error
0,2.0,0.0,0.659183,0.206125,0.134692,1
1,1.0,1.0,0.010913,0.934111,0.054975,0
2,1.0,0.0,0.833222,0.087801,0.078978,1
3,1.0,1.0,0.090163,0.805797,0.104040,0
4,1.0,1.0,0.340482,0.412337,0.247181,0


In [32]:
df_pred['error'].sum()

40

In [34]:
accuracy_score(df_pred['true'], df_pred['pred'])

0.5